In [1]:
#load the libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import html5lib
from pandasql import sqldf



In [2]:
#Load the data
players = pd.read_csv('data/players3.csv')

In [3]:
players['links'] = players['links'].str.replace("profil", "leistungsdatendetails")
players = players[0:250]

In [4]:
# We are telling the website that we are just a browser
headers = {'User-Agent':
'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}


In [5]:
'''
pagelist = []
for i in players['links']:
    pagelist.append(i)
'''

"\npagelist = []\nfor i in players['links']:\n    pagelist.append(i)\n"

In [6]:
# Delete later [:5]
pagelist = list(players['links'])
#pagelist = pagelist[0:2]


In [7]:

dobs = []
tbodies = []
cells = []
pageTree_list = []
pagesoup_list = []
list_player = []
for row in pagelist:
    pageTree = requests.get(row, headers=headers)
    pageTree_list.append(pageTree)
    pageSoup = BeautifulSoup(pageTree.content, 'html.parser') # Getting only Ousman Dembele no Neymar
    
    tbody = pageSoup.select('tbody')[1]
    tbodies.append(tbody)
    
    cell = tbody.select('td')
    cells.append(cell)
    
    dob = pageSoup.select('p')[0].get_text(strip=True)
    dob = re.search('Age:(.+?)\t\t', dob).group(1)
    dobs.append(dob)


In [8]:

date = pd.DataFrame(columns = ['Name', 'Value', 'DOB'])
date['Name'] = players['Players']
date['Value'] = players['values']
date['DOB'] = dobs



In [9]:
data1 = {}
for i, k in zip(players['Players'], players['links']):
        pageTree = requests.get(k, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')
        tbody = pageSoup.select('table')[1]
        data = pd.read_html(str(tbody))    
        data1[i] = data

In [10]:
#df = pd.DataFrame(list(data1.items())) 
#df = pd.DataFrame.from_dict(data1, orient='index')
df = pd.DataFrame([])
for k,v in data1.items():
    x = pd.concat(pd.Series(v).values.tolist())
    x['name'] = k
    df = df.append(x)

#df = pd.concat({k: pd.Series(v) for k, v in data1.items()})

In [11]:
date1 = date
date1.drop_duplicates(subset= ["Name"], inplace=True, ignore_index=True)



In [12]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [13]:
combined = pd.DataFrame(sqldf('SELECT *, * FROM date1, df ON date1.Name=df.name'))

In [14]:
comb = combined.iloc[:, 0:15]

In [15]:
comb = comb[comb.Season.notna()]
comb = comb.reset_index(drop=True)


In [16]:
comb['Season'] = comb['Season'].apply(lambda x: x.split('/')[0],1)

In [17]:
comb['Season'] = comb['Season'].apply(lambda x: x if len(x) == 4 else '20' + x if int(x) < 30 else '19' + x,1)

In [18]:
comb['age'] = 12
for i in range(len(comb)):
    comb['age'][i] = int(comb['Season'][i]) - int(comb['DOB'][i][-4:])

<ipython-input-18-df60c7145027>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comb['age'][i] = int(comb['Season'][i]) - int(comb['DOB'][i][-4:])


In [19]:
comb = comb[comb['age'] >= 19 ]
comb = comb[comb['age'] <= 23 ] 
comb = comb.reset_index(drop=True)


In [20]:
#del comb(['reihenfolge ASC, saison_id DESC, verein_id ASC', 'name', 'Competition'])
comb.drop(['reihenfolge ASC, saison_id DESC, verein_id ASC', 'name', 'Competition', 'Unnamed: 10'], axis=1, inplace=True)

In [21]:
comb.rename(columns={'age': 'Age', 'Competition.1': 'Competition', 'Club': 'Club ID#' , 'Unnamed: 5': 'PPG' , 'PPG': 'Goals' , 'Unnamed: 7': 'Assists' , 'Unnamed: 8': 'Cards' , '/ /': 'Minutes'}, inplace=True)

In [22]:
comb['Cards'] = comb['Cards'].apply(lambda x: x.split('/'),1)

In [23]:
comb['Yellow Cards'] = 0
comb['Double Yellow'] = 0
comb['Red Card'] = 0
for i in range(len(comb)):
    comb['Yellow Cards'][i] = comb['Cards'][i][0]
    comb['Double Yellow'][i] = comb['Cards'][i][1]
    comb['Red Card'][i] = comb['Cards'][i][2]

<ipython-input-23-3f6f60ab24c2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comb['Yellow Cards'][i] = comb['Cards'][i][0]
C:\Users\Mehmet\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-23-3f6f60ab24c2>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comb['Double Yellow'][i] = comb['Cards'][i][1]
<ipython-input-23-3f6f

In [24]:
del comb['Cards']

In [25]:
comb['Value'] = comb['Value'].str.lstrip('£')
comb['Value'] = comb['Value'].str.rstrip('m')

In [26]:
comb.head(50)

,Name,Value,DOB,Season,Competition,Club ID#,PPG,Goals,Assists,Minutes,Age,Yellow Cards,Double Yellow,Red Card
0,Neymar,199.80,"Feb 5, 1992",2011,Club World Cup,2.0,1.5,1,-,180',19,-,-,-
1,Neymar,199.80,"Feb 5, 1992",2013,Champions League,10.0,2.0,4,5,775',21,1,-,-
2,Neymar,199.80,"Feb 5, 1992",2013,Copa del Rey,3.0,2.0,1,-,206',21,1,-,-
3,Neymar,199.80,"Feb 5, 1992",2013,LaLiga,26.0,2.31,9,10,1.736',21,6,-,-
4,Neymar,199.80,"Feb 5, 1992",2013,Supercopa,2.0,1.0,1,-,121',21,1,-,-
5,Neymar,199.80,"Feb 5, 1992",2014,Champions League,12.0,2.5,10,-,1.025',22,1,-,-
6,Neymar,199.80,"Feb 5, 1992",2014,Copa del Rey,6.0,3.0,7,1,503',22,1,-,-
7,Neymar,199.80,"Feb 5, 1992",2014,LaLiga,33.0,2.45,22,9,2.570',22,6,-,-
8,Neymar,199.80,"Feb 5, 1992",2015,Champions League,9.0,2.44,3,5,810',23,2,-,-
9,Neymar,199.80,"Feb 5, 1992",2015,Club World Cup,1.0,3.0,-,2,88',23,1,-,-


In [27]:
# Replace dashes with zeros

comb['Assists'] = comb['Assists'].replace('-', 0)
comb['Goals'] = comb['Goals'].replace('-', 0)
comb['Minutes'] = comb['Minutes'].replace('-', 0)
comb['Yellow Cards'] = comb['Yellow Cards'].str.replace('-', '0')
comb['Double Yellow'] = comb['Double Yellow'].str.replace('-', '0')
comb['Red Card'] = comb['Red Card'].str.replace('-', '0')

In [28]:
comb['Yellow Cards'] = comb['Yellow Cards'].str.strip()
comb['Double Yellow'] = comb['Double Yellow'].str.strip()
comb['Red Card'] = comb['Red Card'].str.strip()

In [29]:
# Covert to numeric 
comb['Goals'] = pd.to_numeric(comb['Goals'])
comb['Assists'] = pd.to_numeric(comb['Assists'])
comb['Yellow Cards'] = pd.to_numeric(comb['Yellow Cards'])
comb['Double Yellow'] = pd.to_numeric(comb['Double Yellow'])
comb['Red Card'] = pd.to_numeric(comb['Red Card'])

In [30]:
# Create a new column goal contribution and assign it the total number of assists and goals

comb['Goal Contribution'] = np.nan
for i in range(len(comb)):
    comb['Goal Contribution'][i] = comb['Goals'][i] + comb['Assists'][i]

<ipython-input-30-f6866753412d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comb['Goal Contribution'][i] = comb['Goals'][i] + comb['Assists'][i]


In [32]:
comb.head(-5)

,Name,Value,DOB,Season,Competition,Club ID#,PPG,Goals,Assists,Minutes,Age,Yellow Cards,Double Yellow,Red Card,Goal Contribution
0,Neymar,199.80,"Feb 5, 1992",2011,Club World Cup,2.0,1.5,1,0,180',19,0,0,0,1.0
1,Neymar,199.80,"Feb 5, 1992",2013,Champions League,10.0,2.0,4,5,775',21,1,0,0,9.0
2,Neymar,199.80,"Feb 5, 1992",2013,Copa del Rey,3.0,2.0,1,0,206',21,1,0,0,1.0
3,Neymar,199.80,"Feb 5, 1992",2013,LaLiga,26.0,2.31,9,10,1.736',21,6,0,0,19.0
4,Neymar,199.80,"Feb 5, 1992",2013,Supercopa,2.0,1.0,1,0,121',21,1,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2907,Jarrod Bowen,19.17,"Dec 20, 1996",2016,FA Cup,-,-,0,0,0,20,0,0,0,0.0
2908,Jarrod Bowen,19.17,"Dec 20, 1996",2016,Premier League,7,000,0,0,177',20,0,0,0,0.0
2909,Jarrod Bowen,19.17,"Dec 20, 1996",2017,Championship,42,1.00,14,2,3.284',21,2,0,0,16.0
2910,Jarrod Bowen,19.17,"Dec 20, 1996",2017,FA Cup,2,3.00,1,1,180',21,0,0,0,2.0


In [33]:
comb.to_csv('data/players4.csv', index=False)